In [141]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign
from IPython.display import display, HTML, Math

In [142]:
delta_b_negative = False
discrete = True
simp = True

In [143]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'l', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, l_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, l_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [144]:
if delta_b_negative:
  a0_eq = Eq(-Delta_b * s_s, Delta_s * s_b)
else:
  a0_eq = Eq(Delta_b * s_s, Delta_s * s_b)
a0_eq

Eq(Delta_b*s_s, Delta_s*s_b)

In [145]:
# if delta_b_negative:
#   a0_eq = Eq(-Delta_b / s_b, Delta_s / s_s)
# else:
#   a0_eq = Eq(Delta_b / s_b, Delta_s / s_s)
# a0_eq

# same result

In [146]:
def deltaBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = (s - l * w) / w # at this point, this value is negative for buying and positive for selling
  # rounding towards zero in all cases, as we are interested in the maximum delta-capacity of a given spot price
  if asset == 'b':
    if delta_b_negative:
      if discrete:
        f = ceiling(f)
    else:
      if discrete:
        f = floor(-f)
      else:
        f = -f

  else:
    if discrete:
      f = floor(f)

  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

l_b - s_b/w_b

In [147]:
deltaBySpot['s']

-l_s + s_s/w_s

In [148]:
solveset(Eq(Delta_b, deltaBySpot['b']), s_b)

{-w_b*(Delta_b - l_b)}

In [149]:
solveset(Eq(Delta_s, deltaBySpot['s']), s_s)

{w_s*(Delta_s + l_s)}

In [150]:
def spotByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  if (not delta_b_negative) and asset == 'b':
    f = w * (l - Delta)
  else:
    f = w * (l + Delta)
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
spotByDelta['b']

w_b*(-Delta_b + l_b)

In [151]:
spotByDelta['s']

w_s*(Delta_s + l_s)

In [152]:
spotByDelta['b'].subs({Delta_b: deltaBySpot['b']})

s_b

In [153]:
spotByDelta['s'].subs({Delta_s: deltaBySpot['s']})

s_s

In [154]:
deltaBySpot['b'].subs({s_b: spotByDelta['b']})

Delta_b

In [155]:
deltaBySpot['s'].subs({s_s: spotByDelta['s']})

Delta_s

In [156]:
a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}).simplify()

Eq(Delta_b*w_s*(Delta_s + l_s), Delta_s*w_b*(-Delta_b + l_b))

In [157]:
def deltaByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = solveset(a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}), Delta)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if discrete:
    if asset == 'b':
      f = floor(f)
    else:
      f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaByDelta = {asset: deltaByDelta_(asset, *all_symbols[asset]) for asset in assets}
deltaByDelta['b']

Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s)

In [158]:
deltaByDelta['s']

-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)

In [164]:
print(deltaByDelta['s'])

-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)


In [159]:
deltaByDelta['s'].subs(Delta_b, l_b) # this would result in a spot price of 0 for b

-l_s

In [160]:
deltaByDelta['s'].subs(Delta_b, l_b - 1).simplify()

l_s*w_s*(1 - l_b)/(l_b*w_s - w_b - w_s)

In [161]:
deltaByDelta['s'].subs({
  Delta_b: 9007199254740990,
  l_b: 9007199254740991,
  l_s: 3518498026389932,
  w_b: 98,
  w_s: 28117773
  })

-222775799575005472578540727551623365410/63315596002644082653793

In [162]:
def spotBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = solveset(a0_eq.subs({Delta_b: deltaBySpot['b'], Delta_s: deltaBySpot['s']}), s)
  # f = list(f)
  # assert len(f) == 1
  # f = f[0]
  if simp:
    f = simplify(f)
  return f

spotBySpot = {asset: spotBySpot_(asset, *all_symbols[asset]) for asset in assets}
spotBySpot['b']

{l_b*s_s*w_b*w_s/(-l_s*w_b*w_s + s_s*w_b + s_s*w_s)}

In [163]:
spotBySpot['s']

{l_s*s_b*w_b*w_s/(-l_b*w_b*w_s + s_b*w_b + s_b*w_s)}